In [2]:
import pandas as pd
import altair as alt

# Time Chart

In [108]:
df = pd.read_csv("./data/final-study-raw-2.csv")
df['trialId'].unique()
conditions = {
    'VIS': ['Covid-Vis-Q1', 'Tennis-Vis-Q1', 'Organizations-Vis-Q1'],
    'Text': ['Covid-Text-Q1', 'Tennis-Text-Q1', 'Organizations-Text-Q1'],
    'Both': ['Covid-Both-Q1', 'Tennis-Both-Q1', 'Organizations-Both-Q1']
}
dataset = {
    'Covid-Text-Q1': "Covid", 
    'Organizations-Vis-Q1': "Organizations", 
    'Tennis-Both-Q1': "Tennis",
    'Organizations-Text-Q1': "Organizations", 
    'Covid-Both-Q1': "Covid", 
    'Tennis-Vis-Q1': "Tennis",
    'Tennis-Text-Q1': "Tennis", 
    'Covid-Vis-Q1': "Covid", 
    'Organizations-Both-Q1': "Organizations",
}
inv_conditions = {v: k for k, values in conditions.items() for v in values}
df['condition'] = df['trialId'].map(inv_conditions)
df['dataset'] = df['trialId'].map(dataset)
df = df[df['condition'].notnull()]
df['duration'] = df['duration'].apply(lambda x: x / 1000 / 60)
# df['answerCorrect'] = df['correctAnswer'] == df['answer']
sub_df = df.groupby(['participantId', 'condition', 'dataset']).aggregate({'duration': 'mean'}).reset_index()


In [112]:
jitterbug = alt.Chart(sub_df).mark_point(filled=True).encode(
    y='condition',
    yOffset="jitter:Q",
    x='duration',
    color=alt.Color('condition', scale=alt.Scale(
        domain=['VIS', 'Text', 'Both'], range=['#A5021D', '#2F88A6', '#635293']), legend=None),
        shape=alt.Shape('dataset', scale=alt.Scale(range=['circle', 'cross', 'square'])),
    tooltip=['condition', 'duration', "participantId"]
).transform_calculate(
    jitter="random()"
).properties(height=200, width=400)

boxes = alt.Chart(sub_df).mark_boxplot().encode(
    y='condition',
    x='duration',
    color=alt.Color('condition', scale=alt.Scale(
        domain=['VIS', 'Text', 'Both'], range=['#A5021D', '#2F88A6', '#635293']), legend=None)
).transform_calculate(jitter="10").properties(height=200, width=400)


jitterbug | boxes

alt.HConcatChart(...)